## Solving the environment

- ALSO: this might be a great example to try supervised learning and show why it doesn't work...???
  - in some ways this is a way better example than frozen lake... because there IS a short-term reward, it's just not what you should look at.
  - with frozen lake there is no short-term reward, so RL seems "obvious"


- REWARD SHAPING 2 types:
  - help it learn
  - actually change what you care about
- examples here: ethical recommendations
- some sort of total reward for the whole episode, not interim rewards
- other stuff

In [1]:
from ray import tune
from envs import BasicRecommender

In [7]:
from ray.rllib.agents.ppo import PPOTrainer
from envs import BasicRecommender

In [8]:
env_config = {
    "num_candidates" : 2,
    "alpha"          : 0.5,
    "seed"           : 42
}

trainer_config = {
    "framework"             : "torch",
    "create_env_on_driver"  : True,
    "seed"                  : 0,
    "lr"                    : 0.001,
    "model"                 : {
        "fcnet_hiddens"     : [64, 64]
    },
    "env_config"            : env_config,
    "output"                : "./data/out1",
    "output_max_file_size"  : 5000000
}

In [9]:
trainer = PPOTrainer(trainer_config, env=BasicRecommender)

2022-07-14 09:55:54,398	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-07-14 09:55:54,400	INFO trainer.py:779 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


In [10]:
for i in range(5):
    trainer.train()
